In [1]:
from IPython.display import HTML
css_str = '<style> \
.jp-icon-warn0 path {fill: var(--jp-warn-color0);} \
.bp3-button-text path { fill: var(--jp-inverse-layout-color3);} \
.jp-icon-brand0 path { fill: var(--jp-brand-color0);} \
text.terms { fill: #616161;} \
</style>'
display(HTML(css_str))

In [1]:
import numpy as np
import pandas as pd
#import datatable
import itertools as it
#from owlready2 import *
import math
import joblib

import pickle
from sklearn.utils import parallel_backend

#para depurar los que letras que no están en el codigo ascii
import unicodedata
import functools
import spacy
#from spacy_stanfordnlp import StanfordNLPLanguage
from sklearn import metrics
#from wiki_dump_reader import Cleaner, iterate
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize,scale


#import gensim
#import gensim.corpora as corpora
#from gensim.models import CoherenceModel
#from gensim.models import HdpModel

#warnings.filterwarningation


from sklearn.svm import SVC
from sklearn.model_selection import *
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
import  h2o

In [5]:
from h2o.estimators import H2OGradientBoostingEstimator,H2ORandomForestEstimator,H2OSupportVectorMachineEstimator

In [6]:

h2o.init(max_mem_size='14G')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "18.0.1.1" 2022-04-22; OpenJDK Runtime Environment Homebrew (build 18.0.1.1+0); OpenJDK 64-Bit Server VM Homebrew (build 18.0.1.1+0, mixed mode, sharing)
  Starting server from /Users/Raul/miniforge3/envs/kge/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/zy/9n6jmtx94qb6l_vcz_gzs41r0000gn/T/tmpsk1w_eo0
  JVM stdout: /var/folders/zy/9n6jmtx94qb6l_vcz_gzs41r0000gn/T/tmpsk1w_eo0/h2o_Raul_started_from_python.out
  JVM stderr: /var/folders/zy/9n6jmtx94qb6l_vcz_gzs41r0000gn/T/tmpsk1w_eo0/h2o_Raul_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.3
H2O_cluster_version_age:,29 days
H2O_cluster_name:,H2O_from_python_Raul_70igzg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,14 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


In [7]:
from hyperopt import fmin, tpe, hp,Trials

def gbm_cv(space):
    """SVC cross validation.
    This function will instantiate a SVC classifier with parameters C and
    gamma. Combined with data and targets this will in turn be used to perform
    cross validation. The result of cross validation is returned.
    Our goal is to find combinations of C and gamma that maximizes the roc_auc
    metric.
    """
    params = {'max_depth': int(space['max_depth']), 'min_rows':int(space['min_rows']),'learn_rate':float(space['eta']),
             'sample_rate':float(space['sample_rate'])
             }
      
    estimator = H2OGradientBoostingEstimator(**params,nfolds=2)
    estimator.train(x=vocab,y='target',training_frame=train, validation_frame = test)
    return estimator.logloss()

def optimize_gbm():
    """Apply Bayesian Optimization to XGB parameters."""
    
    trials = Trials()
    space={"max_depth": hp.uniform('max_depth',4, 40), 'min_rows':hp.uniform('x_min_rows',2,50),'eta':hp.uniform('x_eta',0.01,1),
                'sample_rate':hp.uniform('x_sample_rate',0.1,0.5),'colsample_bytree':hp.uniform('x_colsample_bytree',0.2,0.8)
                 }
    best = fmin(fn=gbm_cv,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

    print("Final result:", best)
    

In [8]:
from hyperopt import fmin, tpe, hp,Trials

def svm_cv(space):
    """SVC cross validation.
    This function will instantiate a SVC classifier with parameters C and
    gamma. Combined with data and targets this will in turn be used to perform
    cross validation. The result of cross validation is returned.
    Our goal is to find combinations of C and gamma that maximizes the roc_auc
    metric.
    """
    params = {'gamma': float(space['gamma'])}
      
    estimator = H2OSupportVectorMachineEstimator(**params,max_iterations=400)
    estimator.train(x=vocab,y='target',training_frame=train)
    return estimator.logloss()

def optimize_svm():
    """Apply Bayesian Optimization to XGB parameters."""
    
    trials = Trials()
    space={"gamma": hp.uniform('x_gamma',0,1)}
    best = fmin(fn=svm_cv,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

    print("Final result:", best)

In [9]:
from hyperopt import fmin, tpe, hp,Trials

def rf_cv(space):
    """SVC cross validation.
    This function will instantiate a SVC classifier with parameters C and
    gamma. Combined with data and targets this will in turn be used to perform
    cross validation. The result of cross validation is returned.
    Our goal is to find combinations of C and gamma that maximizes the roc_auc
    metric.
    """
    params = {'max_depth': int(space['max_depth']), 
              'ntrees': int(space['ntrees']),'min_rows':int(space['min_rows']),
              'sample_rate':float(space['sample_rate'])}
      
    estimator = H2ORandomForestEstimator(**params,nfolds=2)
    estimator.train(x=vocab,y='target',training_frame=train,validation_frame=test)
    return estimator.logloss()

def optimize_rf():
    """Apply Bayesian Optimization to XGB parameters."""
    
    trials = Trials()
    space={"max_depth": hp.uniform('max_depth',4, 40), 
           "ntrees": hp.uniform('x_ntrees',50,400),
           'min_rows':hp.uniform('x_min_rows',2,50),
           'sample_rate':hp.uniform('x_sample_rate',0.1,0.5)}
    best = fmin(fn=rf_cv,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

    print("Final result:", best)

In [10]:
import joblib
entidades=joblib.load('../bbc_objects/entidades_bbc')
topics =joblib.load('../bbc_objects/new_bbc_topics_7')
corpus = joblib.load('../bbc_objects/bbc_processed_final')


FileNotFoundError: [Errno 2] No such file or directory: '../bbc_objects/new_bbc_topics_7'

In [12]:
corpus = joblib.load('../bbc_objects/bbc_processed_final_semantic_2')

In [13]:
vocab = joblib.load('../bbc_objects/bbc_vocab_dtm')

In [14]:
newdtm =joblib.load('../bbc_objects/bbc_dtm')
dtm = newdtm.todense()

In [15]:
from sklearn.preprocessing import normalize
import pandas as pd
dtm_scaled = normalize(dtm)
dtm_scaled = pd.DataFrame(data=dtm_scaled, columns=vocab) ##revisar que está bien generado, si no ir al paso anterior
dtm_scaled['target']=corpus.iloc[:,5]
dtm_scaled.head()

/Users/Raul/miniforge3/envs/kge/lib/python3.10/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


,robinson,match,net,rule,celtic,band,hewitt,coach,arsene,toure,...,grand,group,captain,prime,accuse,row,summer,month,participant,target
0,0.0,0.0,0.016170,0.00000,0.000000,0.0,0.0,0.018632,0.0,0.0,...,0.0,0.0,0.0,0.030193,0.000000,0.0,0.077018,0.002226,0.0,3
1,0.0,0.0,0.000000,0.01333,0.011132,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.004337,0.0,0.000000,0.000000,0.0,1
2,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,5
3,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2
4,0.0,0.0,0.108258,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0


In [17]:
dtm_scaled.to_csv('../bbc_objects/weighteddtmscaled.csv')

In [18]:
frame = h2o.upload_file('../bbc_objects/weighteddtmscaled.csv',sep=',',header=1)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [19]:
frame = frame.drop('C1')

In [20]:
frame['target']=frame['target'].asfactor()
train,test=frame.split_frame(ratios=[.75])

In [21]:

optimize_gbm()

gbm Model Build progress: |                                                     
  0%|                                    | 0/10 [00:00<?, ?trial/s, best loss=?]

/Users/Raul/miniforge3/envs/kge/lib/python3.10/site-packages/h2o/estimators/estimator_base.py:208: RuntimeWarning: Dropping bad and constant columns: [project, man, test, brazilian, ibm]
  warnings.warn(mesg["message"], RuntimeWarning)



█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                           

In [22]:
params={'max_depth': 23, 'learn_rate': 0.7, 'min_rows': 19, 'sample_rate': 0.475}


train

In [23]:
model = H2OGradientBoostingEstimator(**params,seed=123)

In [24]:
model.train(x=vocab,y='target',training_frame=train)

gbm Model Build progress: |

/Users/Raul/miniforge3/envs/kge/lib/python3.10/site-packages/h2o/estimators/estimator_base.py:208: RuntimeWarning: Dropping bad and constant columns: [project, man, test, brazilian, ibm]
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1659803988355_11


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,350.0,142416.0,5.0,23.0,22.697144,6.0,35.0,26.505714




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 9.444831036125864e-08
RMSE: 0.0003073244382753487
LogLoss: 1.908680688097493e-05
Mean Per-Class Error: 0.0
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc
0,,2022-08-06 18:46:16,0.002 sec,0.0,0.857143,1.945910,0.822571,NaN,NaN
1,,2022-08-06 18:46:16,0.098 sec,1.0,0.485663,0.713028,0.202776,NaN,NaN
2,,2022-08-06 18:46:16,0.204 sec,2.0,0.379179,0.463665,0.133977,NaN,NaN
3,,2022-08-06 18:46:16,0.311 sec,3.0,0.309732,0.324631,0.089922,NaN,NaN
4,,2022-08-06 18:46:16,0.415 sec,4.0,0.270125,0.250080,0.074231,NaN,NaN
5,,2022-08-06 18:46:16,0.517 sec,5.0,0.232871,0.195669,0.055522,NaN,NaN
6,,2022-08-06 18:46:16,0.619 sec,6.0,0.191407,0.143087,0.040435,NaN,NaN
7,,2022-08-06 18:46:17,0.726 sec,7.0,0.171470,0.112226,0.029572,NaN,NaN
8,,2022-08-06 18:46:17,0.829 sec,8.0,0.151982,0.091364,0.025347,NaN,NaN
9,,2022-08-06 18:46:17,0.930 sec,9.0,0.135350,0.075748,0.021726,NaN,NaN



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,bell,93.483849,1.000000,0.109809
1,sony,89.068604,0.952770,0.104623
2,berlin,61.991470,0.663125,0.072817
3,dallaglio,58.576580,0.626596,0.068806
4,internet,54.518562,0.583187,0.064039
5,player,31.279087,0.334593,0.036741
6,let,27.087978,0.289761,0.031818
7,handset,20.397808,0.218196,0.023960
8,single,20.254606,0.216664,0.023792
9,van,19.288393,0.206329,0.022657



See the whole table with table.as_data_frame()


In [27]:
performance = model.model_performance(test)

In [28]:
performance


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.12901144069803083
RMSE: 0.3591816263369145
LogLoss: 0.824844654261957
Mean Per-Class Error: 0.16247160645867545
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,Error,Rate
0,35.0,0.0,0.0,0.0,3.0,0.0,2.0,0.125000,5 / 40
1,0.0,115.0,0.0,0.0,2.0,1.0,7.0,0.080000,10 / 125
2,0.0,0.0,22.0,0.0,11.0,3.0,0.0,0.388889,14 / 36
3,0.0,3.0,0.0,83.0,1.0,0.0,0.0,0.045977,4 / 87
4,10.0,4.0,6.0,4.0,71.0,2.0,3.0,0.290000,29 / 100
5,0.0,0.0,2.0,0.0,3.0,59.0,0.0,0.078125,5 / 64
6,1.0,4.0,1.0,5.0,3.0,1.0,101.0,0.129310,15 / 116
7,46.0,126.0,31.0,92.0,94.0,66.0,113.0,0.144366,82 / 568



Top-7 Hit Ratios: 


,k,hit_ratio
0,1,0.855634
1,2,0.955986
2,3,0.989437
3,4,0.998239
4,5,1.000000
5,6,1.000000
6,7,1.000000


In [29]:
model.save_mojo('../bbc_objects/gbm')


'/Users/Raul/doctorado/semantic_oboe/semantic_oboe/bbc_objects/gbm/GBM_model_python_1659803988355_11.zip'

In [30]:
optimize_rf()

drf Model Build progress: |                                                     
  0%|                                    | 0/10 [00:00<?, ?trial/s, best loss=?]

/Users/Raul/miniforge3/envs/kge/lib/python3.10/site-packages/h2o/estimators/estimator_base.py:208: RuntimeWarning: Dropping bad and constant columns: [project, man, test, brazilian, ibm]
  warnings.warn(mesg["message"], RuntimeWarning)



█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                                                                               
█                           

In [31]:
params = {'max_depth':9,'min_rows':5,'ntrees':291,'sample_rate':0.48,"nfolds":10}
rfmodel = H2ORandomForestEstimator(**params,seed=123)


In [32]:
rfmodel.train(x=vocab,y='target',training_frame=train)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1659803988355_22


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,291.0,2037.0,755961.0,6.0,9.0,8.978399,10.0,46.0,24.345116




ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.2391609674943359
RMSE: 0.4890408648511246
LogLoss: 0.6807266047616614
Mean Per-Class Error: 0.14587458682933216
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,Error,Rate
0,112.0,2.0,0.0,1.0,12.0,0.0,0.0,0.118110,15 / 127
1,2.0,366.0,0.0,4.0,3.0,1.0,7.0,0.044386,17 / 383
2,0.0,0.0,54.0,0.0,3.0,19.0,0.0,0.289474,22 / 76
3,0.0,11.0,0.0,256.0,1.0,0.0,0.0,0.044776,12 / 268
4,25.0,16.0,11.0,14.0,146.0,33.0,10.0,0.427451,109 / 255
5,0.0,0.0,1.0,0.0,1.0,222.0,1.0,0.013333,3 / 225
6,3.0,13.0,0.0,6.0,3.0,2.0,296.0,0.083591,27 / 323
7,142.0,408.0,66.0,281.0,169.0,277.0,314.0,0.123718,205 / 1.657



Top-7 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc
0,,2022-08-06 18:55:21,1 min 11.859 sec,0.0,NaN,NaN,NaN,NaN,NaN
1,,2022-08-06 18:55:21,1 min 11.880 sec,1.0,0.569174,1.940413,0.338882,NaN,NaN
2,,2022-08-06 18:55:21,1 min 11.900 sec,2.0,0.538695,1.718924,0.294964,NaN,NaN
3,,2022-08-06 18:55:21,1 min 11.922 sec,3.0,0.528472,1.437343,0.272109,NaN,NaN
4,,2022-08-06 18:55:21,1 min 11.942 sec,4.0,0.521491,1.359152,0.255238,NaN,NaN
5,,2022-08-06 18:55:21,1 min 11.964 sec,5.0,0.514736,1.086308,0.240148,NaN,NaN
6,,2022-08-06 18:55:21,1 min 11.985 sec,6.0,0.510479,0.971587,0.222087,NaN,NaN
7,,2022-08-06 18:55:21,1 min 12.004 sec,7.0,0.502744,0.805393,0.204242,NaN,NaN
8,,2022-08-06 18:55:21,1 min 12.024 sec,8.0,0.498266,0.745805,0.191657,NaN,NaN
9,,2022-08-06 18:55:21,1 min 12.043 sec,9.0,0.494031,0.711128,0.184783,NaN,NaN



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,sony,2949.008545,1.000000,0.021380
1,serve,2324.713379,0.788303,0.016854
2,single,2113.124268,0.716554,0.015320
3,bell,2073.783203,0.703214,0.015034
4,uefa,1899.888550,0.644247,0.013774
5,company,1841.309937,0.624383,0.013349
6,oscar,1787.412598,0.606106,0.012958
7,newcastle,1611.114624,0.546324,0.011680
8,london,1595.339355,0.540975,0.011566
9,tell,1590.162354,0.539219,0.011528



See the whole table with table.as_data_frame()


In [33]:
performance2 = rfmodel.model_performance(test)

In [34]:
performance2


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.2417140432421197
RMSE: 0.4916442242537989
LogLoss: 0.6831727048211105
Mean Per-Class Error: 0.133246921182266
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,Error,Rate
0,32.0,0.0,0.0,1.0,5.0,0.0,2.0,0.200000,8 / 40
1,0.0,117.0,0.0,1.0,0.0,1.0,6.0,0.064000,8 / 125
2,0.0,0.0,30.0,0.0,0.0,6.0,0.0,0.166667,6 / 36
3,0.0,2.0,0.0,85.0,0.0,0.0,0.0,0.022989,2 / 87
4,6.0,3.0,6.0,9.0,64.0,10.0,2.0,0.360000,36 / 100
5,0.0,0.0,1.0,0.0,0.0,63.0,0.0,0.015625,1 / 64
6,2.0,5.0,0.0,4.0,0.0,1.0,104.0,0.103448,12 / 116
7,40.0,127.0,37.0,100.0,69.0,81.0,114.0,0.128521,73 / 568



Top-7 Hit Ratios: 


,k,hit_ratio
0,1,0.871479
1,2,0.966549
2,3,0.989437
3,4,0.994718
4,5,0.998239
5,6,1.000000
6,7,1.000000


In [35]:
rfmodel.save_mojo('../bbc_objects/rfmodel')

'/Users/Raul/doctorado/semantic_oboe/semantic_oboe/bbc_objects/rfmodel/DRF_model_python_1659803988355_22.zip'

In [36]:
h2o.shutdown()

H2O session _sid_a903 closed.


/var/folders/zy/9n6jmtx94qb6l_vcz_gzs41r0000gn/T/ipykernel_31031/2900054706.py:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()
